In [13]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [14]:
# Conjuntos
ciudades_destino = ['Cali', 'Barranquilla', 'Pasto', 'Tunja', 'Chía', 'Manizales']
ciudades_origen = ['Bogotá', 'Medellín']

In [15]:
# Parámetros
demanda = {'Cali': 125, 'Barranquilla': 175, 'Pasto': 225, 'Tunja': 250, 'Chía': 225, 'Manizales': 200}
oferta = {'Bogotá': 550, 'Medellín': 700}

costos_transporte = {
    ('Cali', 'Bogotá'): 0, ('Cali', 'Medellín'): 2.5,
    ('Barranquilla', 'Bogotá'): 2.5, ('Barranquilla', 'Medellín'): 0,
    ('Pasto', 'Bogotá'): 1.6, ('Pasto', 'Medellín'): 2.0,
    ('Tunja', 'Bogotá'): 1.4, ('Tunja', 'Medellín'): 1.0,
    ('Chía', 'Bogotá'): 0.8, ('Chía', 'Medellín'): 1.0,
    ('Manizales', 'Bogotá'): 1.4, ('Manizales', 'Medellín'): 0.8
}

In [16]:
# Crear el modelo
model = ConcreteModel()

# Variables de decisión
model.x = Var(ciudades_destino, ciudades_origen, domain=NonNegativeReals)

In [17]:
# Función objetivo: Minimizar el costo total de transporte
def costo_total(model):
    return sum(costos_transporte[(i, j)] * model.x[i, j] for i in ciudades_destino for j in ciudades_origen)

model.objetivo = Objective(rule=costo_total, sense=minimize)

In [18]:
# Restricción de demanda: cada ciudad debe recibir al menos su demanda
def demanda_satisfecha(model, i):
    return sum(model.x[i, j] for j in ciudades_origen) >= demanda[i]

model.restriccion_demanda = Constraint(ciudades_destino, rule=demanda_satisfecha)

In [19]:
# Restricción de oferta: cada ciudad de origen no puede enviar más productos de los que tiene disponibles
def oferta_disponible(model, j):
    return sum(model.x[i, j] for i in ciudades_destino) <= oferta[j]

model.restriccion_oferta = Constraint(ciudades_origen, rule=oferta_disponible)

In [20]:
# Resolver el modelo
solver = SolverFactory('glpk')
resultado = solver.solve(model)

# Mostrar los resultados de las variables de decisión
model.display()

Model unknown

  Variables:
    x : Size=12, Index={Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}*{Bogotá, Medellín}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Barranquilla', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Barranquilla', 'Medellín') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Cali', 'Bogotá') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Cali', 'Medellín') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Chía', 'Bogotá') :     0 : 200.0 :  None : False : False : NonNegativeReals
                ('Chía', 'Medellín') :     0 :  25.0 :  None : False : False : NonNegativeReals
             ('Manizales', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('Manizales', 'Medellín') :     0 : 200.0 :  None : False : False : NonNegativeReals
                 ('Pasto

In [21]:
# Cargar duales
model.dual = Suffix(direction=Suffix.IMPORT)

# Resolver el modelo con duales
resultado = solver.solve(model)

In [22]:
#Mostrar duales de las restricciones
print("\nAnálisis de Sensibilidad:")
for i in ciudades_destino:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.restriccion_demanda[i]]}")

for j in ciudades_origen:
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.restriccion_oferta[j]]}")



Análisis de Sensibilidad:
Dual de la restricción de demanda en Cali: 0.2
Dual de la restricción de demanda en Barranquilla: 0.0
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chía: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogotá: -0.2
Dual de la restricción de oferta en Medellín: 0.0


In [23]:
# Mover 50 toneladas de oferta de Medellín a Bogotá y repetir el análisis de sensibilidad
print("\nMover 50 toneladas de oferta de Medellín a Bogotá\n")

oferta['Bogotá'] += 50
oferta['Medellín'] -= 50

# Resolver nuevamente con la nueva distribución de oferta
resultado = solver.solve(model)
model.display()



Mover 50 toneladas de oferta de Medellín a Bogotá

Model unknown

  Variables:
    x : Size=12, Index={Cali, Barranquilla, Pasto, Tunja, Chía, Manizales}*{Bogotá, Medellín}
        Key                          : Lower : Value : Upper : Fixed : Stale : Domain
          ('Barranquilla', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Barranquilla', 'Medellín') :     0 : 175.0 :  None : False : False : NonNegativeReals
                  ('Cali', 'Bogotá') :     0 : 125.0 :  None : False : False : NonNegativeReals
                ('Cali', 'Medellín') :     0 :   0.0 :  None : False : False : NonNegativeReals
                  ('Chía', 'Bogotá') :     0 : 200.0 :  None : False : False : NonNegativeReals
                ('Chía', 'Medellín') :     0 :  25.0 :  None : False : False : NonNegativeReals
             ('Manizales', 'Bogotá') :     0 :   0.0 :  None : False : False : NonNegativeReals
           ('Manizales', 'Medellín') :     0 : 200.0 :  None : False

In [24]:
# Mostrar duales de las nuevas restricciones
print("\nAnálisis de Sensibilidad después del cambio:")
for i in ciudades_destino:
    print(f"Dual de la restricción de demanda en {i}: {model.dual[model.restriccion_demanda[i]]}")

for j in ciudades_origen:
    print(f"Dual de la restricción de oferta en {j}: {model.dual[model.restriccion_oferta[j]]}")



Análisis de Sensibilidad después del cambio:
Dual de la restricción de demanda en Cali: 0.2
Dual de la restricción de demanda en Barranquilla: 0.0
Dual de la restricción de demanda en Pasto: 1.8
Dual de la restricción de demanda en Tunja: 1.0
Dual de la restricción de demanda en Chía: 1.0
Dual de la restricción de demanda en Manizales: 0.8
Dual de la restricción de oferta en Bogotá: -0.2
Dual de la restricción de oferta en Medellín: 0.0
